In [1]:
import utils
inp = utils.get_input(2019, 9)[:-1]
print(inp)

Found cached input for 2019 day 9
1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,3,1,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1102,1,30,1010,1102,1,38,1008,1102,1,0,1020,1102,22,1,1007,1102,26,1,1015,1102,31,1,1013,1102,1,27,1014,1101,0,23,1012,1101,0,37,1006,1102,735,1,1028,1102,1,24,1009,1102,1,28,1019,1102,20,1,1017,1101,34,0,1001,1101,259,0,1026,1101,0,33,1018,1102,1,901,1024,1101,21,0,1016,1101,36,0,1011,1102,730,1,1029,1101,1,0,1021,1102,1,509,1022,1102,39,1,1005,1101,35,0,1000,1102,1,506,1023,1101,0,892,1025,1101,256,0,1027,1101,25,0,1002,1102,1,29,1004,1102,32,1,1003,109,9,1202,-3,1,63,1008,63,39,63,1005,63,205,1001,64,1,64,1106,0,207,4,187,1002,64,2,64,109,-2,1208,-4,35,63,1005,63,227,1001,64,1,64,1105,1,229,4,213,1002,64,2,64,109,5,1206,8,243,4,235,1106,0,247,1001,64,1,64,1002,64,2,64,109,14,2106,0,1,1105,1,265,4,253,1001,64,1,64,1002,

## machine

In [2]:
from collections import defaultdict

mem = [1,2,3]
d = defaultdict(int, enumerate(mem))
d[0]
d[4]

0

In [58]:
import operator
from functools import partial
from itertools import repeat, chain
from collections import namedtuple, defaultdict
from copy import deepcopy


class IntCode:
    Op = namedtuple('Op', ['func', 'num_params', 'write_param', 'resolve_modes'])

    def __init__(self, program, pointer=0, rel_base=0, **kwargs):
        self.mem = defaultdict(int, enumerate(program))
#         print(self.mem)
        self.pointer = pointer
        self.rel_base = rel_base
        self.state = "started"
        self.output = None
        self._kwargs = kwargs
        self._ops = {
            1: IntCode.Op(partial(IntCode._math, operator.add), 3, True, True),
            2: IntCode.Op(partial(IntCode._math, operator.mul), 3, True, True),
            3: IntCode.Op(IntCode._inp, 1, True, True),
            4: IntCode.Op(IntCode._out, 1, False, True),
            5: IntCode.Op(partial(IntCode._jump, lambda x: x != 0), 2, False, True),
            6: IntCode.Op(partial(IntCode._jump, lambda x: x == 0), 2, False, True),
            7: IntCode.Op(partial(IntCode._math, operator.lt), 3, True, True),
            8: IntCode.Op(partial(IntCode._math, operator.eq), 3, True, True),
            9: IntCode.Op(IntCode._base, 1, False, True),
            99: IntCode.Op(IntCode._halt, 0, False, False),
        }

    def run(self, inputs=None):
        inputs = iter(inputs or [])

        while self.state != "halted":
            instruction = self.mem[self.pointer]
            opcode, modes = self._opcode(instruction), self._modes(instruction)
            op = self._ops[opcode]
            params = [self.mem[i] for i in range(self.pointer + 1, self.pointer + op.num_params + 1)]
            raw_params = list(params)

            if op.resolve_modes:
                if opcode in [1, 2, 7, 8]:
                    params[0] = self._resolve(self.mem, self.rel_base, params[0], modes[0], True)
                    params[1] = self._resolve(self.mem, self.rel_base, params[1], modes[1], True)
                    params[2] = self._resolve(self.mem, self.rel_base, params[2], modes[2], False)
                elif opcode in [5, 6]:
                    params[0] = self._resolve(self.mem, self.rel_base, params[0], modes[0], True)
                    params[1] = self._resolve(self.mem, self.rel_base, params[1], modes[1], True)
                elif opcode in [3]:
                    params[0] = self._resolve(self.mem, self.rel_base, params[0], modes[0], False)
                elif opcode in [4, 9]:
                    params[0] = self._resolve(self.mem, self.rel_base, params[0], modes[0], True)
                    
                    
#                 wparam = -int(op.write_param) or len(params)# if len(params) > 1 else len(params)
# #                 if instruction == 1102:
# #                     print(opcode, modes, params, wparam)
#                 args = zip(repeat(self.mem), repeat(self.rel_base), params[:wparam], modes[:wparam])
#                 params[:wparam] = map(lambda t: IntCode._resolve(*t), args)
#             if instruction == 203:
#                 print(self.mem)
            if self._kwargs.get('_debug', False):
                print(f"{instruction:5d}", raw_params, f"{opcode:2d}", modes, list(params), end='')

            orig_pointer = self.pointer
            self.mem, self.pointer, self.rel_base, output, self.state = \
                op.func(self.mem, self.pointer, self.rel_base, *params, inputs=inputs, **self._kwargs)
            self.output = output if output is not None else self.output

            if self._kwargs.get('_debug', False):
                print(f" -> {output, self.pointer, self.rel_base}")

            if (output is not None) or (self.state == "blocked"):
                yield self

            if self.pointer == orig_pointer:
                self.pointer = self.pointer + op.num_params + 1
        yield self

    @staticmethod
    def _resolve(mem, rel_base, val, mode, param=True):
        if int(mode) == 0:
            return mem[val] if param else val
        elif int(mode) == 2:
            assert (rel_base + val) >= 0
            return mem[val + rel_base] if param else (val + rel_base)
        else:
            return val

    @staticmethod
    def _opcode(instruction):
        return int(str(instruction)[-2:])

    @staticmethod
    def _modes(instruction):
        return f"{instruction:05d}"[:3][::-1]

    @staticmethod
    def _math(func, mem, pointer, rel_base, a, b, out, **kwargs):
#         _mem = deepcopy(mem)
        _mem = mem
        _mem[out] = int(func(a, b))
        return _mem, pointer, rel_base, None, "running"

    @staticmethod
    def _jump(func, mem, pointer, rel_base, cond, val, **kwargs):
        return mem, (val if func(cond) else pointer), rel_base, None, "running"

    @staticmethod
    def _base(mem, pointer, rel_base, val, **kwargs):
        return mem, pointer, (rel_base + val), None, "running"

    @staticmethod
    def _inp(mem, pointer, rel_base, out, inputs, **kwargs):
        try:
#             _mem = deepcopy(mem)
            _mem = mem
            _mem[out] = int(next(inputs))
            return _mem, pointer, rel_base, None, "running"
        except StopIteration:
            return mem, pointer, rel_base, None, "blocked"

    @staticmethod
    def _out(mem, pointer, rel_base, val, **kwargs):
#         if kwargs.get('_debug', False):
#             print(f"out: {val}")
        return mem, pointer, rel_base, val, "running"

    @staticmethod
    def _halt(mem, pointer, rel_base, *args, **kwargs):
        return mem, pointer, rel_base, None, "halted"


In [61]:
program = list(map(int, inp.split(',')))
# program = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
# program = [1102,34915192,34915192,7,4,7,99,0]
# program = [104,1125899906842624,99]
# program = [109,19,204,-34,99]
vm = IntCode(program, _debug=False).run([2])
while True:
    out = next(vm)
#     print(out.pointer)
#     print(out.state)
#     print(out.rel_base)
    if out.state == "halted":
        break
#     print(out.rel_base)
    print(out.output)
#     print(sorted(out.mem))
#     print(out.pointer)
# print(out.output)

76642
